# Object condensation using CLUE3D

Goal:
- start with layer-clusters (x,y,z,e)
- run edgeconv
- collapse to tracksters
- run edgeconv
- fully connected
- query edges
- output

In [8]:
import torch
import torch.nn as nn
import torch_geometric.transforms as T
import torch_geometric.utils as geo_utils

from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
import sklearn.metrics as metrics

from reco.model import EdgeConvNet
from torch_geometric.nn import EdgeConv, DynamicEdgeConv, global_mean_pool

from reco.learn import train_edge_pred, test_edge_pred
from reco.dataset import PointCloudSet
from reco.loss import FocalLoss
from reco.training import split_geo_train_test

data_root = "data"
ds_name = "MultiParticle"
raw_dir = f"/Users/ecuba/data/{ds_name}"

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [9]:
transform = T.Compose([T.NormalizeFeatures()])

ds = PointCloudSet(
    ds_name,
    data_root,
    raw_dir,
    transform=transform, # todo: z-axis transformation
    N_FILES=50,
)

positive_edge_fr = float(sum(ds.data.y) / len(ds.data.y))
print(f"Positive edge ratio: {positive_edge_fr:.3f}") 
train_dl, test_dl = split_geo_train_test(ds, batch_size=1)
ds.data

Positive edge ratio: 0.362
Train set: 4500, Test set: 500


Data(x=[10684122, 4], edge_index=[2, 2695051], y=[2695051], trackster_index=[10684122])

In [13]:
class PointCloudNet(nn.Module):
    def __init__(self, input_dim=4, output_dim=1, aggr='add', dropout=0.2):
        super(PointCloudNet, self).__init__()

        lc_hdim1 = 32        
        lc_hdim2 = 64

        tr_hdim1 = 64
        tr_hdim2 = 64

        fc_hdim = 128

        k=4

        # EdgeConv on LC
        self.lc_conv1 = DynamicEdgeConv(nn=EdgeConvNet(input_dim, lc_hdim1), aggr=aggr, k=k)
        self.lc_conv2 = DynamicEdgeConv(nn=EdgeConvNet(lc_hdim1, lc_hdim2), aggr=aggr, k=k)

        # EdgeConv on Tracksters
        self.trackster_conv1 = DynamicEdgeConv(nn=EdgeConvNet(lc_hdim2, tr_hdim1), aggr=aggr, k=k)
        self.trackster_conv2 = DynamicEdgeConv(nn=EdgeConvNet(tr_hdim1, tr_hdim2), aggr=aggr, k=k)

        # Edge features from node embeddings for classification
        self.edgenetwork = nn.Sequential(
            nn.Linear(2 * tr_hdim2, fc_hdim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(fc_hdim, output_dim),
            nn.Sigmoid()
        )
            
    def forward(self, X, query, tr_index):        

        # tr index has gaps due to wrong reindexation on pytorch geometric with batch_size > 1
        # last = -1
        # idx = -1
        # tridx2lc = {}
        # l_tr_index = tr_index.tolist()
        # new_idx = [0] * len(l_tr_index)
        # for i, tr_i in enumerate(l_tr_index):
        #     if tr_i != last:
        #         last = tr_i
        #         idx += 1
        #         tridx2lc[idx] = []

        #     new_idx[i] = idx
        #     tridx2lc[idx].append(i)

        # build knn edges within each trackster
        # lc_edges = []   # knn edges witin a trackster

        # Convolution on layer-clusters
        H = self.lc_conv1(X)    # (BATCH_SIZE, N_LC, 4) -> (BATCH_SIZE, N_LC, 32)
        H = self.lc_conv2(H)    # (BATCH_SIZE, N_LC, 32) -> (BATCH_SIZE, N_LC, 64)

        # Condensation into tracksters using pooling: (max, mean, add, topK, self-attention)
        TX = global_mean_pool(H, tr_index)  # (BATCH_SIZE, N_LC, 64) -> (BATCH_SIZE, N_TR, 64)

        # Convolution on tracksters
        H = self.trackster_conv1(TX)    # (BATCH_SIZE, N_TR, 64) -> (BATCH_SIZE, N_TR, 64)
        H = self.trackster_conv2(H)     # (BATCH_SIZE, N_TR, 64) -> (BATCH_SIZE, N_TR, 64)

        src, dst = query
        q_edges = torch.cat([H[src], H[dst]], dim=-1)   # (BATCH_SIZE, N_TR, 64) -> (BATCH_SIZE, Q_EDGES, 128)
        return self.edgenetwork(q_edges).squeeze(-1)

In [14]:
model = PointCloudNet(input_dim=ds.data.x.shape[1])
epochs = 10

# loss_func = F.binary_cross_entropy_with_logits
# alpha - percentage of negative edges
loss_func = FocalLoss(alpha=positive_edge_fr, gamma=2)

model = model.to(device)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, epochs, eta_min=1e-3)

for epoch in range(epochs):

    train_loss, train_true, train_pred = train_edge_pred(
        model,
        device,
        optimizer,
        loss_func,
        train_dl
    )
    
    train_acc = metrics.accuracy_score(train_true, (train_pred > 0.5).astype(int))
    scheduler.step()

    if epoch % 1 == 0:
        test_loss, test_true, test_pred = test_edge_pred(model, device, loss_func, test_dl)
        test_acc = metrics.accuracy_score(test_true, (test_pred > 0.5).astype(int))
        print(
            f"Epoch {epoch}:",
            f"\ttrain loss:{train_loss:.2f}\ttrain acc: {train_acc:.3f}",
            f"\t test loss:{test_loss:.2f} \t test acc: {test_acc:.3f}"
        )

Epoch 0: 	train loss:339.83	train acc: 0.655 	 test loss:702.53 	 test acc: 0.382
Epoch 1: 	train loss:318.41	train acc: 0.668 	 test loss:577.36 	 test acc: 0.398
Epoch 2: 	train loss:307.33	train acc: 0.674 	 test loss:671.88 	 test acc: 0.405
Epoch 3: 	train loss:299.45	train acc: 0.679 	 test loss:1542.02 	 test acc: 0.403
Epoch 4: 	train loss:292.91	train acc: 0.683 	 test loss:1039.65 	 test acc: 0.404
Epoch 5: 	train loss:287.49	train acc: 0.687 	 test loss:1988.55 	 test acc: 0.405
Epoch 6: 	train loss:283.05	train acc: 0.690 	 test loss:2640.19 	 test acc: 0.404
Epoch 7: 	train loss:279.06	train acc: 0.693 	 test loss:3652.29 	 test acc: 0.395
Epoch 8: 	train loss:275.50	train acc: 0.696 	 test loss:4190.82 	 test acc: 0.390
Epoch 9: 	train loss:272.45	train acc: 0.698 	 test loss:3704.50 	 test acc: 0.394
